# Run pipeline with bash 

You will need to install Salmon, fastQC and multiQC before running the commands below. Otherwise, you can create the conda environment, activate it and run the notebook then, as explained [here](create_conda_env.ipynb). 



In [1]:
import os, shutil
pwd = os.getcwd()
os.getcwd()
pipeline_folder = "rnaseq-nf" 
work_dir = pwd + "/../" + pipeline_folder
os.chdir(work_dir)
# os.getcwd()

In [2]:
!salmon index --threads 1 -t ./data/ggal/ggal_1_48850000_49020000.Ggal71.500bpflank.fa -i index

Version Info: ### PLEASE UPGRADE SALMON ###
### A newer version of salmon with important bug fixes and improvements is available. ####
###
The newest version, available at https://github.com/COMBINE-lab/salmon/releases
contains new features, improvements, and bug fixes; please upgrade at your
earliest convenience.
###
Sign up for the salmon mailing list to hear about new versions, features and updates at:
https://oceangenomics.com/subscribe
###
index ["index"] did not previously exist  . . . creating it
[2020-01-28 08:30:06.876] [jLog] [info] building index
out : index
[2020-01-28 08:30:06.876] [puff::index::jointLog] [info] Running fixFasta

[Step 1 of 4] : counting k-mers

[2020-01-28 08:30:06.895] [puff::index::jointLog] [info] Replaced 0 non-ATCG nucleotides
[2020-01-28 08:30:06.895] [puff::index::jointLog] [info] Clipped poly-A tails from 0 transcripts
wrote 1 cleaned references
seqHash 256 : c7b06752fa0748ecfa05ab97e8460b69c408b6d5fedf9de3d0d53b05570cf942
seqHash 512 : ed859a805d

In [3]:
!salmon quant --threads 1 --libType=U -i index -1 ./data/ggal/ggal_gut_1.fq -2 ./data/ggal/ggal_gut_2.fq  -o results_salmon
!rm -rf index

Version Info: ### PLEASE UPGRADE SALMON ###
### A newer version of salmon with important bug fixes and improvements is available. ####
###
The newest version, available at https://github.com/COMBINE-lab/salmon/releases
contains new features, improvements, and bug fixes; please upgrade at your
earliest convenience.
###
Sign up for the salmon mailing list to hear about new versions, features and updates at:
https://oceangenomics.com/subscribe
###
### salmon (mapping-based) v1.0.0
### [ program ] => salmon 
### [ command ] => quant 
### [ threads ] => { 1 }
### [ libType ] => { U }
### [ index ] => { index }
### [ mates1 ] => { ./data/ggal/ggal_gut_1.fq }
### [ mates2 ] => { ./data/ggal/ggal_gut_2.fq }
### [ output ] => { results_salmon }
Logs will be written to results_salmon/logs
[2020-01-28 08:30:07.460] [jointLog] [info] Fragment incompatibility prior below threshold.  Incompatible fragments will be ignored.
[2020-01-28 08:30:07.460] [jointLog] [info] Usage of --validateMappings impli

In [4]:
if not os.path.exists("results_fastqc"):
    os.makedirs("results_fastqc")
    
!fastqc -o results_fastqc -f fastq -q ./data/ggal/ggal_gut_*.fq



In [5]:
if not os.path.exists("multiqc_in"):
    os.makedirs("multiqc_in")

if not os.path.exists("results_multiqc"):
    os.makedirs("results_multiqc")

!cp results_fastqc/* multiqc_in
!cp -rf results_salmon/* multiqc_in
!cp multiqc/* multiqc_in
!echo "custom_logo: $PWD/multiqc_in/logo.png" >> multiqc_in/multiqc_config.yaml

pwd = os.getcwd()
work_dir = pwd + "/multiqc_in"
os.chdir(work_dir)
os.getcwd()

!multiqc -v . --outdir ../results_multiqc/


/home/kadomu/conda_env_nxf/rnaseq-nf-58fed063cfbc6adc6b42861c7e7fae6d/lib/python3.6/site-packages/multiqc/utils/config.py:45: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  configs = yaml.load(f)
/home/kadomu/conda_env_nxf/rnaseq-nf-58fed063cfbc6adc6b42861c7e7fae6d/lib/python3.6/site-packages/multiqc/utils/config.py:51: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  sp = yaml.load(f)
[2020-01-28 08:30:13,422] multiqc                                            [DEBUG  ]  No MultiQC config found: /home/kadomu/conda_env_nxf/rnaseq-nf-58fed063cfbc6adc6b42861c7e7fae6d/lib/python3.6/site-packages/multiqc_config.yaml
[2020-01-28 08:30:13,422] multiqc                                            [DEBUG  ]  No MultiQC config found: /home/kadomu/.multiqc_config.yaml
/

[2020-01-28 08:30:14,996] multiqc.modules.salmon.salmon                      [INFO   ]  Found 1 meta reports
[2020-01-28 08:30:14,996] multiqc.modules.salmon.salmon                      [INFO   ]  Found 1 fragment length distributions
[2020-01-28 08:30:15,000] multiqc                                            [DEBUG  ]  No samples found: kallisto
[2020-01-28 08:30:15,003] multiqc                                            [DEBUG  ]  No samples found: slamdunk
[2020-01-28 08:30:15,006] multiqc                                            [DEBUG  ]  No samples found: star
[2020-01-28 08:30:15,008] multiqc                                            [DEBUG  ]  No samples found: hisat2
[2020-01-28 08:30:15,010] multiqc                                            [DEBUG  ]  No samples found: tophat
[2020-01-28 08:30:15,013] multiqc                                            [DEBUG  ]  No samples found: bowtie2
[2020-01-28 08:30:15,015] multiqc                                            [DEBUG 

In [6]:
shutil.rmtree(work_dir)